https://github.com/dbouchabou/Fully-Convolutional-Network-Smart-Homes/tree/master

In [ ]:
from google.colab import drive
drive.mount('/content/drive' , force_remount = True)

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
cd "/content/drive/MyDrive/PAMAP2_Dataset"

/content/drive/MyDrive/PAMAP2_Dataset


In [ ]:
data = pd.read_pickle('pamap2_for_lstm.pkl')

In [ ]:
data.shape

(1942872, 43)

In [ ]:
data = data[data['id'] != 109]
train_data = data[(data["id"] != 103) & (data["id"] != 102)]
test_data = data[(data["id"] == 102)]
val_data = data[(data["id"] == 103)]

In [ ]:
test_data

In [ ]:
def segment_activities(df):
    activitiesSeq = []

    ponentialIndex = df.activity_id.ne(df.activity_id.shift())

    ii = np.where(ponentialIndex == True)[0]

    for i,end in enumerate(ii):
        if i > 0 :

          dftmp = df[ii[i-1]:end]
          activitiesSeq.append(dftmp)
    return activitiesSeq

In [ ]:
train_activitySequences = segment_activities(train_data)
val_activitySequences = segment_activities(val_data)
test_activitySequences = segment_activities(test_data)

In [ ]:
test_activitySequences

#### with my columns

In [ ]:
def generate_sentence(df2):
    sentence = ""

    # Define the column names you want to extract values from
    columns_of_interest = ['heart_rate', 'hand_temperature', 'hand_3D_acceleration_16_x', 'hand_3D_acceleration_16_y',
                           'hand_3D_acceleration_16_z', 'hand_3D_acceleration_6_x', 'hand_3D_acceleration_6_y',
                           'hand_3D_acceleration_6_z', 'hand_3D_gyroscope_x', 'hand_3D_gyroscope_y',
                           'hand_3D_gyroscope_z', 'hand_3D_magnetometer_x', 'hand_3D_magnetometer_y',
                           'hand_3D_magnetometer_z', 'chest_temperature', 'chest_3D_acceleration_16_x',
                           'chest_3D_acceleration_16_y', 'chest_3D_acceleration_16_z', 'chest_3D_acceleration_6_x',
                           'chest_3D_acceleration_6_y', 'chest_3D_acceleration_6_z', 'chest_3D_gyroscope_x',
                           'chest_3D_gyroscope_y', 'chest_3D_gyroscope_z', 'chest_3D_magnetometer_x',
                           'chest_3D_magnetometer_y', 'chest_3D_magnetometer_z', 'ankle_temperature',
                           'ankle_3D_acceleration_16_x', 'ankle_3D_acceleration_16_y', 'ankle_3D_acceleration_16_z',
                           'ankle_3D_acceleration_6_x', 'ankle_3D_acceleration_6_y', 'ankle_3D_acceleration_6_z',
                           'ankle_3D_gyroscope_x', 'ankle_3D_gyroscope_y', 'ankle_3D_gyroscope_z',
                           'ankle_3D_magnetometer_x', 'ankle_3D_magnetometer_y', 'ankle_3D_magnetometer_z']

    # Iterate over the columns of interest
    for column in columns_of_interest:
        # Get the value for the current column
        value = df2[column].values[0]  # Directly access the single value in the column

        # Add column name and value to the sentence
        sentence += "{}{}".format(column, value)

        # Add a space if it's not the last column
        if column != columns_of_interest[-1]:
            sentence += " "

    return sentence

In [ ]:
def sequencesToSentences(activitySequences):
	sentences = []
	label_sentences = []

	for i in range(len(activitySequences)):

		sentence = generate_sentence(activitySequences[i])

		sentences.append(sentence)
		label_sentences.append(activitySequences[i].activity_id.values[0])

	return sentences, label_sentences

In [ ]:
train_sentences, train_label_sentences = sequencesToSentences(train_activitySequences)
val_sentences, val_label_sentences = sequencesToSentences(val_activitySequences)
test_sentences, test_label_sentences = sequencesToSentences(test_activitySequences)

In [ ]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
combined_sentences = train_sentences + val_sentences + test_sentences
tokenizer.fit_on_texts(combined_sentences)

# Tokenize train, validation, and test sets
train_indexed_sentences = tokenizer.texts_to_sequences(train_sentences)
val_indexed_sentences = tokenizer.texts_to_sequences(val_sentences)
test_indexed_sentences = tokenizer.texts_to_sequences(test_sentences)

In [ ]:
tokenizer1 = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
tokenizer1.fit_on_texts(train_sentences)
word_index = tokenizer1.word_index
train_indexed_sentences = tokenizer1.texts_to_sequences(train_sentences)

tokenizer2 = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
tokenizer2.fit_on_texts(val_sentences)
word_index = tokenizer2.word_index
val_indexed_sentences = tokenizer2.texts_to_sequences(val_sentences)

tokenizer3 = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
tokenizer3.fit_on_texts(test_sentences)
word_index = tokenizer3.word_index
test_indexed_sentences = tokenizer3.texts_to_sequences(test_sentences)

In [ ]:
len(tokenizer.word_index)

3961

In [ ]:
def slidingWindow(sequence,winSize,step=1):

    try: it = iter(sequence)
    except TypeError:
        raise Exception("**ERROR** sequence must be iterable.")
    if not ((type(winSize) == type(0)) and (type(step) == type(0))):
        raise Exception("**ERROR** type(winSize) and type(step) must be int.")
    if step > winSize:
        raise Exception("**ERROR** step must not be larger than winSize.")

    numOfChunks = int(((len(sequence)-winSize)/step)+1)

    # Do the work
    if winSize > len(sequence):
        yield sequence[0:len(sequence)]
    else:
        for i in range(0,numOfChunks*step,step):
            yield sequence[i:i+winSize]

In [ ]:
X_train_windowed = []
Y_train_windowed = []
winSize = 50
step = 1

for i,s in enumerate(train_indexed_sentences):
	chunks = slidingWindow(s,winSize,step)
	for chunk in chunks:
		X_train_windowed.append(chunk)
		Y_train_windowed.append(train_label_sentences[i])

X_val_windowed = []
Y_val_windowed = []
winSize = 50
step = 1

for i,s in enumerate(val_indexed_sentences):
	chunks = slidingWindow(s,winSize,step)
	for chunk in chunks:
		X_val_windowed.append(chunk)
		Y_val_windowed.append(val_label_sentences[i])


X_test_windowed = []
Y_test_windowed = []
winSize = 50
step = 1

for i,s in enumerate(test_indexed_sentences):
	chunks = slidingWindow(s,winSize,step)
	for chunk in chunks:
		X_test_windowed.append(chunk)
		Y_test_windowed.append(test_label_sentences[i])

In [ ]:
train_padded_windows = pad_sequences(X_train_windowed)
test_padded_windows = pad_sequences(X_test_windowed)
val_padded_windows = pad_sequences(X_val_windowed)

In [ ]:
Y_train_windowed = np.array(Y_train_windowed)
Y_test_windowed = np.array(Y_test_windowed)
Y_val_windowed = np.array(Y_val_windowed)

np.save("{}_{}_tpadded_x.npy".format("lstm",winSize), train_padded_windows)
np.save("{}_{}_tpadded_y.npy".format("lstm",winSize), Y_train_windowed)
np.save("{}_{}_ttpadded_x.npy".format("lstm",winSize), test_padded_windows)
np.save("{}_{}_ttpadded_y.npy".format("lstm",winSize), Y_test_windowed)
np.save("{}_{}_vpadded_x.npy".format("lstm",winSize), val_padded_windows)
np.save("{}_{}_vpadded_y.npy".format("lstm",winSize), Y_val_windowed)

#### Split on subjects:
  subjects 105 for val
  106 for test

In [ ]:
all_data = pd.concat(activitySequences)

In [ ]:
unique_subject_ids = all_data['id'].unique()

train_data = []
val_data = []
test_data = []

for subject_id in unique_subject_ids:
    subject_data = all_data[all_data['id'] == subject_id]

    if subject_id == 105:
        val_data.append(subject_data)
    elif subject_id == 106:
        test_data.append(subject_data)
    else:
        train_data.append(subject_data)


In [ ]:
train_data = pd.concat(train_data)
val_data = pd.concat(val_data)
test_data = pd.concat(test_data)

In [ ]:
unique_train_ids = val_data['id'].unique()
print(unique_train_ids)

[105]


In [ ]:
def generate_sentence(df2):
    sentence = ""

    # Define the column names you want to extract values from
    columns_of_interest = ['heart_rate', 'hand_temperature', 'hand_3D_acceleration_16_x', 'hand_3D_acceleration_16_y',
                           'hand_3D_acceleration_16_z', 'hand_3D_acceleration_6_x', 'hand_3D_acceleration_6_y',
                           'hand_3D_acceleration_6_z', 'hand_3D_gyroscope_x', 'hand_3D_gyroscope_y',
                           'hand_3D_gyroscope_z', 'hand_3D_magnetometer_x', 'hand_3D_magnetometer_y',
                           'hand_3D_magnetometer_z', 'chest_temperature', 'chest_3D_acceleration_16_x',
                           'chest_3D_acceleration_16_y', 'chest_3D_acceleration_16_z', 'chest_3D_acceleration_6_x',
                           'chest_3D_acceleration_6_y', 'chest_3D_acceleration_6_z', 'chest_3D_gyroscope_x',
                           'chest_3D_gyroscope_y', 'chest_3D_gyroscope_z', 'chest_3D_magnetometer_x',
                           'chest_3D_magnetometer_y', 'chest_3D_magnetometer_z', 'ankle_temperature',
                           'ankle_3D_acceleration_16_x', 'ankle_3D_acceleration_16_y', 'ankle_3D_acceleration_16_z',
                           'ankle_3D_acceleration_6_x', 'ankle_3D_acceleration_6_y', 'ankle_3D_acceleration_6_z',
                           'ankle_3D_gyroscope_x', 'ankle_3D_gyroscope_y', 'ankle_3D_gyroscope_z',
                           'ankle_3D_magnetometer_x', 'ankle_3D_magnetometer_y', 'ankle_3D_magnetometer_z']

    # Iterate over the columns of interest
    for column in columns_of_interest:
        # Get the value for the current column
        value = df2[column]  # Directly access the single value in the column

        # Add column name and value to the sentence
        sentence += "{}{}".format(column, value)

        # Add a space if it's not the last column
        if column != columns_of_interest[-1]:
            sentence += " "

    return sentence

In [ ]:
def sequencesToSentences(activitySequences):
    sentences = []
    label_sentences = []

    for _, row in activitySequences.iterrows():
        sentence = generate_sentence(row)
        sentences.append(sentence)
        label_sentences.append(row['activity_id'])

    return sentences, label_sentences

In [ ]:
train_sentences, train_label_sentences = sequencesToSentences(train_data)

In [ ]:
test_sentences, test_label_sentences = sequencesToSentences(test_data)

In [ ]:
val_sentences, val_label_sentences = sequencesToSentences(val_data)

In [ ]:
len(val_sentences)

272442

In [ ]:
train_sentences_array = np.array(train_sentences)
train_label_sentences_array = np.array(train_label_sentences)

# Define the file paths
sentences_file = "train_sentences.txt"
labels_file = "train_label_sentences.txt"

# Save the sentences and labels to separate text files
np.savetxt(sentences_file, train_sentences_array, fmt="%s")
np.savetxt(labels_file, train_label_sentences_array, fmt="%s")

In [ ]:
tokenizer1 = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
tokenizer1.fit_on_texts(train_sentences)
word_index1 = tokenizer1.word_index
train_indexed_sentences = tokenizer1.texts_to_sequences(train_sentences)

In [ ]:
tokenizer2 = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
tokenizer2.fit_on_texts(test_sentences)
word_index2 = tokenizer2.word_index
test_indexed_sentences = tokenizer2.texts_to_sequences(test_sentences)

In [ ]:
tokenizer3 = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
tokenizer3.fit_on_texts(val_sentences)
word_index3 = tokenizer3.word_index
val_indexed_sentences = tokenizer3.texts_to_sequences(val_sentences)

#### original

In [ ]:
def generate_sentence(df2):

    sentence = ""

    val = ""

    #extract sensors list
    sensors = df2.sensor.values

    values = df2.value.values

    #iterate on sensors list
    for i in range(len(sensors)):

        val = values[i]

        if i == len(sensors) - 1:
            sentence += "{}{}".format(sensors[i],val)
        else:
            sentence += "{}{} ".format(sensors[i],val)

    return sentence

#### with my columns

In [ ]:
def generate_sentence(df2):
    sentence = ""

    # Define the column names you want to extract values from
    columns_of_interest = ['heart_rate', 'hand_temperature', 'hand_3D_acceleration_16_x', 'hand_3D_acceleration_16_y',
                           'hand_3D_acceleration_16_z', 'hand_3D_acceleration_6_x', 'hand_3D_acceleration_6_y',
                           'hand_3D_acceleration_6_z', 'hand_3D_gyroscope_x', 'hand_3D_gyroscope_y',
                           'hand_3D_gyroscope_z', 'hand_3D_magnetometer_x', 'hand_3D_magnetometer_y',
                           'hand_3D_magnetometer_z', 'chest_temperature', 'chest_3D_acceleration_16_x',
                           'chest_3D_acceleration_16_y', 'chest_3D_acceleration_16_z', 'chest_3D_acceleration_6_x',
                           'chest_3D_acceleration_6_y', 'chest_3D_acceleration_6_z', 'chest_3D_gyroscope_x',
                           'chest_3D_gyroscope_y', 'chest_3D_gyroscope_z', 'chest_3D_magnetometer_x',
                           'chest_3D_magnetometer_y', 'chest_3D_magnetometer_z', 'ankle_temperature',
                           'ankle_3D_acceleration_16_x', 'ankle_3D_acceleration_16_y', 'ankle_3D_acceleration_16_z',
                           'ankle_3D_acceleration_6_x', 'ankle_3D_acceleration_6_y', 'ankle_3D_acceleration_6_z',
                           'ankle_3D_gyroscope_x', 'ankle_3D_gyroscope_y', 'ankle_3D_gyroscope_z',
                           'ankle_3D_magnetometer_x', 'ankle_3D_magnetometer_y', 'ankle_3D_magnetometer_z']

    # Iterate over the columns of interest
    for column in columns_of_interest:
        # Get the value for the current column
        value = df2[column].values[0]  # Directly access the single value in the column

        # Add column name and value to the sentence
        sentence += "{}{}".format(column, value)

        # Add a space if it's not the last column
        if column != columns_of_interest[-1]:
            sentence += " "

    return sentence

In [ ]:
def sequencesToSentences(activitySequences):
	sentences = []
	label_sentences = []

	for i in range(len(activitySequences)):

		sentence = generate_sentence(activitySequences[i])

		sentences.append(sentence)
		label_sentences.append(activitySequences[i].activity_id.values[0])

	return sentences, label_sentences

In [ ]:
sentences, label_sentences = sequencesToSentences(activitySequences)

sentences indexization

In [ ]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
indexed_sentences = tokenizer.texts_to_sequences(sentences)

In [ ]:
len(tokenizer.word_index)

4038

In [ ]:
indexed_sentences

sliding windows

In [ ]:
def slidingWindow(sequence,winSize,step=1):

    try: it = iter(sequence)
    except TypeError:
        raise Exception("**ERROR** sequence must be iterable.")
    if not ((type(winSize) == type(0)) and (type(step) == type(0))):
        raise Exception("**ERROR** type(winSize) and type(step) must be int.")
    if step > winSize:
        raise Exception("**ERROR** step must not be larger than winSize.")

    numOfChunks = int(((len(sequence)-winSize)/step)+1)

    # Do the work
    if winSize > len(sequence):
        yield sequence[0:len(sequence)]
    else:
        for i in range(0,numOfChunks*step,step):
            yield sequence[i:i+winSize]

In [ ]:
X_windowed = []
Y_windowed = []
winSize = 50
step = 1

for i,s in enumerate(indexed_sentences):
	chunks = slidingWindow(s,winSize,step)
	for chunk in chunks:
		X_windowed.append(chunk)
		Y_windowed.append(label_sentences[i])

In [ ]:
padded_windows = pad_sequences(X_windowed)

In [ ]:
Y_windowed = np.array(Y_windowed)

## Save files ##
print("STEP 8: save sliding windows and labels")
np.save("{}_{}_padded_x.npy".format("lstm",winSize), padded_windows)
np.save("{}_{}_padded_y.npy".format("lstm",winSize), Y_windowed)

STEP 8: save sliding windows and labels


In [ ]:
padded_windows.shape

(14593, 50)

In [ ]:
padded_windows[0].shape

(50,)

### main

In [ ]:
train_padded_windows = np.load("{}_{}_tpadded_x.npy".format("lstm", winSize))
Y_train_windowed = np.load("{}_{}_tpadded_y.npy".format("lstm", winSize))
test_padded_windows = np.load("{}_{}_ttpadded_x.npy".format("lstm", winSize))
Y_test_windowed = np.load("{}_{}_ttpadded_y.npy".format("lstm", winSize))
val_padded_windows = np.load("{}_{}_vpadded_x.npy".format("lstm", winSize))
Y_val_windowed = np.load("{}_{}_vpadded_y.npy".format("lstm", winSize))

In [ ]:
Y_train_windowed.shape

(11141,)

In [ ]:
x_train = train_padded_windows
y_train = Y_train_windowed
x_test = test_padded_windows
y_test = Y_test_windowed
x_val = val_padded_windows
y_val = Y_val_windowed

#### FCN embedded

In [ ]:
pip install tensorflow

In [ ]:
class LSTMModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(hidden_dim)
        self.fc = tf.keras.layers.Dense(output_dim, activation='softmax')

    def call(self, inputs):
        embedded = self.embedding(inputs)
        lstm_out = self.lstm(embedded)
        output = self.fc(lstm_out)
        return output

In [ ]:
class LSTMModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm1 = tf.keras.layers.LSTM(hidden_dim, return_sequences=True)
        self.dropout = tf.keras.layers.Dropout(0.5)  # Example dropout rate
        self.lstm2 = tf.keras.layers.LSTM(hidden_dim)
        self.fc = tf.keras.layers.Dense(output_dim, activation='softmax')

    def call(self, inputs):
        embedded = self.embedding(inputs)
        lstm_out1 = self.lstm1(embedded)
        lstm_out2 = self.lstm2(self.dropout(lstm_out1))
        output = self.fc(lstm_out2)
        return output

In [ ]:
flat_x_train = [token for sublist in x_train for token in sublist]
vocab_size = len(set(flat_x_train))
embedding_dim = 100
hidden_dim = 128
output_dim = 25

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

Vocabulary size: 3962


In [ ]:
model = LSTMModel(vocab_size+5, embedding_dim, hidden_dim, output_dim)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=6, batch_size=40, validation_data=(x_val, y_val))

Epoch 1/6
279/279 [==============================] - 67s 221ms/step - loss: 2.1490 - accuracy: 0.2345 - val_loss: 2.8487 - val_accuracy: 0.2651
Epoch 2/6
279/279 [==============================] - 60s 215ms/step - loss: 0.7050 - accuracy: 0.7122 - val_loss: 5.2563 - val_accuracy: 0.2163
Epoch 3/6
279/279 [==============================] - 61s 219ms/step - loss: 0.1287 - accuracy: 0.9688 - val_loss: 6.7418 - val_accuracy: 0.1981
Epoch 4/6
279/279 [==============================] - 64s 230ms/step - loss: 0.1060 - accuracy: 0.9699 - val_loss: 6.7241 - val_accuracy: 0.2068
Epoch 5/6
279/279 [==============================] - 61s 219ms/step - loss: 0.0229 - accuracy: 0.9949 - val_loss: 7.4108 - val_accuracy: 0.2047
Epoch 6/6
279/279 [==============================] - 61s 218ms/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 7.8816 - val_accuracy: 0.2047


#### predict

In [ ]:
predictions = model.predict(x_test)

43/43 [==============================] - 6s 135ms/step


In [ ]:
predictions.shape

(1807, 25)

In [ ]:
import numpy as np

class_predictions = np.argmax(predictions, axis=1)

In [ ]:
class_predictions

array([16,  6,  3, ...,  2, 24,  6])

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, class_predictions)
print("Accuracy:", accuracy)

Accuracy: 0.2046613255644574
